<a href="https://colab.research.google.com/github/andrewwgordon/lang-chain-testbed/blob/main/lang_chain_testbed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU \
  transformers \
  sentence-transformers \
  faiss-gpu \
  accelerate \
  einops \
  langchain \
  xformers \
  bitsandbytes \
  pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.3/270.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.7 MB/s eta 0:00:0

In [2]:
# Model configuration
#
# Sentence transformer model
SENTENCE_TRANSFORMER_MODEL = 'all-MiniLM-L6-v2'
# Batch size (on GPU)
EMBEDDINGS_BATCH_SIZE = 32
# Character chunk size for text splitting
CHARACTER_SPLITTER_CHUNK_SIZE = 1000
# Character chunk overlap size
CHARACTER_SPLITTER_OVERLAP_SIZE = 0
# Large Language Model
LARGE_LANGUAGE_MODEL = 'stabilityai/StableBeluga-7B'
# Text generation task
TRANSFORMER_PIPELINE_TASK = 'text-generation'
# LLM temperature
LLM_TEMPERATURE = 0.0
# LLM max tokens
LLM_MAX_TOKENS = 512
# LLM repetition penalty
LLM_REPETITION_PENALTY = 1.1
# Chain type
CHAIN_TYPE = 'stuff'

In [3]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = f'sentence-transformers/{SENTENCE_TRANSFORMER_MODEL}'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': EMBEDDINGS_BATCH_SIZE}
)

In [4]:
!mkdir docs
!wget -P ./docs http://support.diamond-air.at/fileadmin/uploads/files/after_sales_support/DA40%20New%20Generation/Airplane_Maintenance_Manual/Basic_Manual/60215-r3-DA40-NG-AMM-complete.pdf
!wget -P ./docs http://support.diamond-air.at/fileadmin/uploads/files/after_sales_support/DA40%20New%20Generation/Airplane_Flight_Manual/Basic_Manual/60115e-DA40-NG-AFM-r3-complete.pdf

--2023-08-07 09:03:36--  http://support.diamond-air.at/fileadmin/uploads/files/after_sales_support/DA40%20New%20Generation/Airplane_Maintenance_Manual/Basic_Manual/60215-r3-DA40-NG-AMM-complete.pdf
Resolving support.diamond-air.at (support.diamond-air.at)... 80.120.70.48
Connecting to support.diamond-air.at (support.diamond-air.at)|80.120.70.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38502967 (37M) [application/pdf]
Saving to: ‘./docs/60215-r3-DA40-NG-AMM-complete.pdf’

60215-r3-DA40-NG-AM 100%[===================>]  36.72M  2.23MB/s    in 22s     

2023-08-07 09:03:59 (1.68 MB/s) - ‘./docs/60215-r3-DA40-NG-AMM-complete.pdf’ saved [38502967/38502967]

--2023-08-07 09:03:59--  http://support.diamond-air.at/fileadmin/uploads/files/after_sales_support/DA40%20New%20Generation/Airplane_Flight_Manual/Basic_Manual/60115e-DA40-NG-AFM-r3-complete.pdf
Resolving support.diamond-air.at (support.diamond-air.at)... 80.120.70.48
Connecting to support.diamond-air.at (s

In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./docs/")
pages = loader.load_and_split()

In [6]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name=SENTENCE_TRANSFORMER_MODEL)

In [7]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=CHARACTER_SPLITTER_CHUNK_SIZE, chunk_overlap=CHARACTER_SPLITTER_OVERLAP_SIZE)
docs = text_splitter.split_documents(pages)

In [8]:
from langchain.vectorstores import FAISS
index = FAISS.from_documents(docs, embedding_function)

In [9]:
from torch import cuda, bfloat16
import transformers

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth = 'HF_AUTH_TOKEN'
model_config = transformers.AutoConfig.from_pretrained(
    LARGE_LANGUAGE_MODEL,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    LARGE_LANGUAGE_MODEL,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


In [10]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    LARGE_LANGUAGE_MODEL,
    use_auth_token=hf_auth
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [11]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task=TRANSFORMER_PIPELINE_TASK,
    # we pass model parameters here too
    temperature=LLM_TEMPERATURE,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=LLM_MAX_TOKENS,  # mex number of tokens to generate in the output
    repetition_penalty=LLM_REPETITION_PENALTY  # without this output begins repeating
)

In [12]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)

In [13]:
from langchain.chains import RetrievalQA

query = RetrievalQA.from_chain_type(
    llm=llm, chain_type=CHAIN_TYPE,
    retriever=index.as_retriever()
)

In [39]:
query('In the checklist for normal operating procedures detail the steps of the pre-flight inspection cabin check.')

{'query': 'In the checklist for normal operating procedures detail the steps of the pre-flight inspection cabin check.',
 'result': ' The pre-flight inspection cabin check includes checking the cabin documents, ensuring the cabin is clean, undamaged, and all electrical equipment is off.'}